In [1]:
from transformer_chat_model.chat_models import TransformerChatModel

In [2]:
from transformers import BitsAndBytesConfig
import torch
from langchain_core.tools import tool

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, AIMessageChunk

In [4]:
pretrained_model_name_or_path = "Qwen/Qwen2.5-3B-Instruct"
# Create your quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
chat_model = TransformerChatModel(
    pretrained_model_name_or_path=pretrained_model_name_or_path,
    quantization_config = bnb_config,
    max_new_tokens = 1024,
    device = "auto"
    )

Loading Model ...



Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

In [5]:
@tool
def check_weather(location:str) -> str:
    """Check weather at a location"""
    return "Rain" if location == "Hannoi" else "Cloudy"
@tool
def factorial(num:int)->int:
    """Calculate fatorial of number"""
    return 1

In [6]:
llm_with_tools = chat_model.bind_tools(tools=[check_weather, factorial])

In [12]:
llm_with_tools.debug=True
llm_with_tools.invoke("what the weather in NYC and by the way, what is factorial of 4, 7 and 20?")

=========================DEBUG - TOOL CALLS=========================
<tool_call>
[
    {
        "name": "check_weather",
        "arguments": {"location": "NYC"}
    },
    {
        "name": "factorial",
        "arguments": {"num": 4}
    },
    {
        "name": "factorial",
        "arguments": {"num": 7}
    },
    {
        "name": "factorial",
        "arguments": {"num": 20}
    }
]
</tool_call>
=========================DEBUG - PROMPT=========================
<|im_start|>system
You are a helpful assistant.You have access to the following tools:
**check_weather**:
	Check weather at a location
	params:
		location (string)

**factorial**:
	Calculate fatorial of number
	params:
		num (integer)


Whenever you need to use tools, you MUST use <tool_call> tags to wrap your tool calls at one time.
Each tool call MUST have exactly one key 'name' and one key 'arguments'.
All tool calls must be in this format exactly:
<tool_call>
[
    {
        "name": tool_name,
        "arguments": {"pa

AIMessage(content='', additional_kwargs={}, response_metadata={}, id='lc_run--019c7200-e399-7930-9f37-e946e27dc523-0', tool_calls=[{'name': 'check_weather', 'args': {'location': 'NYC'}, 'id': 'lc_141d8525-e647-42c5-a5ca-b3af69f447bf', 'type': 'tool_call'}, {'name': 'factorial', 'args': {'num': 4}, 'id': 'lc_12c114ea-7d45-46ad-9537-80c7c36bb394', 'type': 'tool_call'}, {'name': 'factorial', 'args': {'num': 7}, 'id': 'lc_13fdc4ce-94a8-4a3a-b011-85d6dfa5ff9e', 'type': 'tool_call'}, {'name': 'factorial', 'args': {'num': 20}, 'id': 'lc_ef7d9094-a8eb-480c-947c-b2d44006279e', 'type': 'tool_call'}], invalid_tool_calls=[])

In [13]:
from langchain.agents import create_agent

In [14]:
agent = create_agent(
    model = chat_model,
    tools = [check_weather, factorial]
)

In [17]:
agent.invoke({"messages":[HumanMessage("what the weather in NYC and by the way, what is factorial of 4, 7 and 20?")]})

{'messages': [HumanMessage(content='what the weather in NYC and by the way, what is factorial of 4, 7 and 20?', additional_kwargs={}, response_metadata={}, id='9f4ecc61-75e9-4c6f-9548-7ff8eef405b6'),
  AIMessage(content='', additional_kwargs={}, response_metadata={}, id='lc_run--019c7204-3d7e-7351-8867-23e72f33cf72-0', tool_calls=[{'name': 'check_weather', 'args': {'location': 'New York, NY'}, 'id': 'lc_d8b7d214-644d-47a9-a80a-a6d00a3f403c', 'type': 'tool_call'}, {'name': 'factorial', 'args': {'num': 4}, 'id': 'lc_200bf9a2-47a4-4347-9c25-8ec8585b6086', 'type': 'tool_call'}, {'name': 'factorial', 'args': {'num': 7}, 'id': 'lc_c504c722-ad5e-495d-9e38-d5478e83ab06', 'type': 'tool_call'}, {'name': 'factorial', 'args': {'num': 20}, 'id': 'lc_057d7d4f-0c66-44e5-a832-a9fe7fb455e0', 'type': 'tool_call'}], invalid_tool_calls=[]),
  ToolMessage(content='Cloudy', name='check_weather', id='3083de97-dab6-4cf8-8f11-97f63649e791', tool_call_id='lc_d8b7d214-644d-47a9-a80a-a6d00a3f403c'),
  ToolMessage